In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import cartopy
import pickle
import xarray as xr
from matplotlib import colors
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import matplotlib as mpl
from scipy.stats import pearsonr
import glob

In [ ]:
mpl.rc('font', family='DejaVu Serif') 
mpl.rcParams['mathtext.rm']='DejaVu Serif'
mpl.rcParams['mathtext.fontset'] = 'custom'
# mpl.rc('font', serif='Helvetica Neue') 

In [ ]:
root_dir = "/p/lustre3/shiduan/ForceSMIP"
files = glob.glob(root_dir + '/Training/' + 'Amon' + '/' + 'pr' + '/*')
models = [p.split('/')[-1] for p in files]
models = sorted(models)
print(models)
model_number = [50, 25, 30, 50, 30]

In [ ]:
def get_model(model, stand, month):
    path = '/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/' + \
        str(1979)+'_2022/model_'+str(model)+'/'+'pr' + \
        '-CMIP-metrics-stand-'+str(stand)+'-month-'+str(month)+'-unforced-False-joint-False'
    with open(path, 'rb') as pfile:
        results = pickle.load(pfile)
    return results
def get_model_gpcp(model, stand, month):
    path = '/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/' + \
        str(1979)+'_2022/model_'+str(model)+'_GPCP/'+'pr' + \
        '-metrics-stand-'+str(stand)+'-month-'+str(month)+'-unforced-False-joint-False'
    with open(path, 'rb') as pfile:
        results = pickle.load(pfile)
    return results
def get_model_mswep(model, stand, month):
    path = '/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/' + \
        str(1979)+'_2022/model_'+str(model)+'_MSWEP/'+'pr' + \
        '-metrics-stand-'+str(stand)+'-month-'+str(month)+'-unforced-False-joint-False'
    with open(path, 'rb') as pfile:
        results = pickle.load(pfile)
    return results

In [ ]:
results = get_model_mswep('CanESM5', True, True)

In [ ]:
results.keys()

In [ ]:
for model in models:
    results1 = get_model(model, stand=False, month=False)
    results2 = get_model(model, stand=True, month=False)
    plt.plot(results1['sn'], color='black')
    plt.plot(results2['sn'], color='red')
    results1 = get_model_gpcp(model, stand=False, month=False)
    results2 = get_model_gpcp(model, stand=True, month=False)
    plt.plot(results1['sn_obs'], color='blue', linestyle='--')
    plt.plot(results2['sn_obs'], color='blue')
    plt.title(model)
    plt.show()

In [ ]:
for model in models:
    results1 = get_model(model, stand=False, month=False)
    results2 = get_model(model, stand=True, month=False)
    plt.plot(results1['sn'], color='black')
    plt.plot(results2['sn'], color='red')
    results1 = get_model_mswep(model, stand=False, month=False)
    results2 = get_model_mswep(model, stand=True, month=False)
    plt.plot(results1['sn_obs'], color='blue', linestyle='--')
    plt.plot(results2['sn_obs'], color='blue')
    plt.title(model)
    plt.show()

# Combine

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
pinatubo = np.datetime64('1991-06')
agung = np.datetime64('1963-02')
elchichon = np.datetime64('1982-04')
sns_colors = sns.color_palette('muted', 3)

In [ ]:
def plot_eof_mode(solver_list_month, solver_list, row, col, fig, gs, norm, cmap='BrBG', left=True, skip=False, neofs=0, space=3, std=False):
    for ind, i in enumerate([0, 3]):
        eof = solver_list_month[i].eofs().isel(mode=neofs).transpose('lat', 'lon')
        pc = solver_list_month[i].pcs().isel(mode=neofs)
        m, b = np.polyfit(np.arange(len(pc)), pc, deg=1)
        if m<0:
            print(i, 'reversed')
            eof = -eof
        variance = str(int(solver_list_month[i].varianceFraction().data[neofs].round(2)*100))+'%'
        ax = fig.add_subplot(gs[row+ind*2:row+ind*2+2, col], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        title = months[i]+' '+variance
        if std:
            title = months[i]+' StdA '+variance
        ax.set_title(title, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)  
    for ind, i in enumerate([6, 9]):
        eof = solver_list_month[i].eofs().isel(mode=neofs).transpose('lat', 'lon')
        pc = solver_list_month[i].pcs().isel(mode=neofs)
        m, b = np.polyfit(np.arange(len(pc)), pc, deg=1)
        if m<0:
            print(i, 'reversed')
            eof = -eof
        variance = str(int(solver_list_month[i].varianceFraction().data[neofs].round(2)*100))+'%'
        ax = fig.add_subplot(gs[row-ind*2+2:row-ind*2+4, col+1], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        title = months[i]+' '+variance
        if std:
            title = months[i]+' StdA '+variance
        ax.set_title(title, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)  
    pc_all = []
    for i in range(12):
        pc_month = solver_list_month[i].pcs().isel(mode=0)
        m, b = np.polyfit(np.arange(len(pc_month)), pc_month, deg=1)
        if m<0:
            pc_month = -pc_month
        pc_all.append(pc_month)
    pc_all = xr.concat(pc_all, dim='time').sortby('time')
    norm_pc_month = (pc_all-pc_all.min())/(pc_all.max()-pc_all.min())*2-1
    
    eof = solver_list[0].eofs().isel(mode=neofs).transpose('lat', 'lon')
    pc = solver_list[0].pcs().isel(mode=neofs)
    norm_pc = (pc-pc.min())/(pc.max()-pc.min())*2-1
    m, b = np.polyfit(np.arange(len(pc)), norm_pc, deg=1)
    print(m)
    if m<0:
        eof = -eof
        print('flip')
        norm_pc = -norm_pc
    variance = str(int(solver_list[0].varianceFraction().data[neofs].round(2)*100))+'%'
    return eof, variance, norm_pc, norm_pc_month

In [ ]:
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates

def plot_model(model):
    with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
        model_solver =  pickle.load(pfile)
    with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
        model_month_solver =  pickle.load(pfile)
    with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
        model_stand_solver =  pickle.load(pfile)
    with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
        model_month_stand_solver =  pickle.load(pfile)
    fig = plt.figure(figsize=(12, 7))
    pc1_time = model_solver[0].pcs().indexes['time'].to_datetimeindex()
    vmin = -0.08
    vmax = 0.08
    levels = np.linspace(vmin, vmax, 21)
    norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    gs = GridSpec(8, 6, figure=fig, width_ratios=[1, 0.1, 1, 1, 0.2, 1, ], wspace=0.1, hspace=1.5)

    eof_allm, variance_allm, norm_pc1, norm_pc_month1 = plot_eof_mode(model_month_solver, model_solver, 0, 2, fig, gs, norm)
    ax = fig.add_subplot(gs[1:3, 0], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof_allm.lon, eof_allm.lat, eof_allm, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.set_title('S-INV '+variance_allm, fontsize=12)
    eof_allm, variance_allm, norm_pc2, norm_pc_month2 = plot_eof_mode(model_month_stand_solver, model_stand_solver, 4, 2, fig, gs, norm, std=True)
    ax = fig.add_subplot(gs[5:7, 0], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof_allm.lon, eof_allm.lat, eof_allm, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.set_title('S-INV StdA '+variance_allm, fontsize=12)
    # PC
    ax = fig.add_subplot(gs[0:2, 5])
    m, b = np.polyfit(np.arange(len(norm_pc1)), norm_pc1, deg=1)
    ax.plot(pc1_time, norm_pc1, label='S-INV Anomaly '+f"{m:.2e}", color=sns_colors[0])
    ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
    m, b = np.polyfit(np.arange(len(norm_pc1)), norm_pc2, deg=1)
    ax.plot(pc1_time, norm_pc2, label='S-INV StdAnomaly '+f"{m:.2e}", color=sns_colors[1])
    m, b = np.polyfit(np.arange(len(norm_pc1)), norm_pc_month1, deg=1)
    ax.plot(pc1_time, norm_pc_month1, label='S-VAR Anomaly '+f"{m:.2e}", color=sns_colors[0], linestyle='dotted')
    m, b = np.polyfit(np.arange(len(norm_pc1)), norm_pc_month2, deg=1)
    ax.plot(pc1_time, norm_pc_month2, label='S-VAR StdAnomaly '+f"{m:.2e}", color=sns_colors[1], linestyle='dotted')
    ax.set_ylabel('Normalized PC')
    ax.xaxis.set_major_locator(MaxNLocator(nbins=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.set_title(model, fontsize=12)
    plt.legend(ncol=1, bbox_to_anchor=(1.02, .9))
    # S/N
    sn_start_time = np.datetime64('1984-01')
    sn_time = [sn_start_time+np.timedelta64(i, 'M') for i in range(444)]
    ax = fig.add_subplot(gs[2:4, 5])
    results1 = get_model(model, stand=False, month=False)
    results2 = get_model(model, stand=True, month=False)
    ax.plot(sn_time, results1['sn'], color=sns_colors[0], label='S-INV Anomaly')
    ax.plot(sn_time, results2['sn'], color=sns_colors[1], label='S-INV StdAnomaly')
    results1 = get_model(model, stand=False, month=True)
    results2 = get_model(model, stand=True, month=True)
    ax.plot(sn_time, results1['sn'], color=sns_colors[0], label='S-VAR Anomaly', linestyle='dotted')
    ax.plot(sn_time, results2['sn'], color=sns_colors[1], label='S-VAR StdAnomaly', linestyle='dotted')
    ax.set_ylabel('SNR')
    ax.xaxis.set_major_locator(MaxNLocator(nbins=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.set_title(model, fontsize=12)
    # S/N for obs
    ax = fig.add_subplot(gs[4:6, 5])
    results1 = get_model_gpcp(model, stand=False, month=False)
    results2 = get_model_gpcp(model, stand=True, month=False)
    ax.plot(sn_time, results1['sn'], color=sns_colors[0], label='S-INV Anomaly')
    ax.plot(sn_time, results2['sn'], color=sns_colors[1], label='S-INV StdAnomaly')
    results1 = get_model_gpcp(model, stand=False, month=True)
    results2 = get_model_gpcp(model, stand=True, month=True)
    ax.plot(sn_time, results1['sn'], color=sns_colors[0], label='S-VAR Anomaly', linestyle='dotted')
    ax.plot(sn_time, results2['sn'], color=sns_colors[1], label='S-VAR StdAnomaly', linestyle='dotted')
    ax.set_ylabel('SNR')
    ax.set_title('GPCP', fontsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    ax = fig.add_subplot(gs[6:8, 5])
    results1 = get_model_mswep(model, stand=False, month=False)
    results2 = get_model_mswep(model, stand=True, month=False)
    ax.plot(sn_time, results1['sn'], color=sns_colors[0], label='S-INV Anomaly')
    ax.plot(sn_time, results2['sn'], color=sns_colors[1], label='S-INV StdAnomaly')
    results1 = get_model_mswep(model, stand=False, month=True)
    results2 = get_model_mswep(model, stand=True, month=True)
    ax.plot(sn_time, results1['sn'], color=sns_colors[0], label='S-VAR Anomaly', linestyle='dotted')
    ax.plot(sn_time, results2['sn'], color=sns_colors[1], label='S-VAR StdAnomaly', linestyle='dotted')
    ax.set_ylabel('SNR')
    ax.set_title('MSWEP', fontsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    cbar_ax = fig.add_axes([0, 0.35, 0.01, 0.3])
    cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
    cbar_ax.yaxis.set_ticks_position('left')
    fig.subplots_adjust(left=0.05, right=0.94, top=0.95, bottom=0.05, hspace=1.2, wspace=0.4)
    fig.suptitle(model, fontsize=14, y=1.02)
    fig.text(x=.05, y=.84, s='a', fontsize=14, weight='bold')
    fig.text(x=.29, y=.96, s='b', fontsize=14, weight='bold')
    fig.text(x=.05, y=.35, s='c', fontsize=14, weight='bold')
    fig.text(x=.29, y=.47, s='d', fontsize=14, weight='bold')
    fig.text(x=.72, y=.96, s='e', fontsize=14, weight='bold')
    fig.text(x=.72, y=.71, s='f', fontsize=14, weight='bold')
    fig.text(x=.72, y=.47, s='g', fontsize=14, weight='bold')
    fig.text(x=.72, y=.22, s='h', fontsize=14, weight='bold')
    plt.savefig(model+'.png', dpi=120, bbox_inches='tight')
    plt.show()

In [ ]:
plot_model(model='CanESM5')

In [ ]:
plot_model(model='CESM2')

In [ ]:
plot_model(model='MIROC6')

In [ ]:
plot_model(model='MPI-ESM1-2-LR')

In [ ]:
plot_model(model='MIROC-ES2L')

In [ ]:
model = 'MIROC-ES2L'
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    model_solver =  pickle.load(pfile)
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    model_month_solver =  pickle.load(pfile)
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    model_stand_solver =  pickle.load(pfile)
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    model_month_stand_solver =  pickle.load(pfile)
plt.plot(model_month_stand_solver[3].pcs().isel(mode=0))
plt.show()
plt.plot(model_stand_solver[0].pcs().isel(mode=0))

In [ ]:
model = 'MPI-ESM1-2-LR'
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    model_solver =  pickle.load(pfile)
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    model_month_solver =  pickle.load(pfile)
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    model_stand_solver =  pickle.load(pfile)
with open('/g/g92/shiduan/lustre2/ForceSMIP/EOF/modes_all/1979_2022/model_'+model+'/pr-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    model_month_stand_solver =  pickle.load(pfile)
plt.plot(model_month_stand_solver[3].pcs().isel(mode=0))
plt.show()
plt.plot(model_stand_solver[0].pcs().isel(mode=0))